In [1]:
import os
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [13]:
# Parameters
train_folder = "images/test_v2/test"
train_csv = "images/written_name_test_v2.csv"
test_folder = "images/validation_v2/validation"
test_csv = "images/written_name_validation_v2.csv"

In [14]:
# Step 1: Load the Data
def load_data(image_folder, csv_file, image_size=(64, 64)):
    # Read CSV
    data = pd.read_csv(csv_file)
    
    # Containers for images and labels
    images = []
    labels = []
    
    # Iterate over the CSV entries
    for _, row in data.iterrows():
        img_path = os.path.join(image_folder, row['FILENAME'])
        
        # Load and preprocess the image
        if os.path.exists(img_path):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale
            img = cv2.resize(img, image_size)  # Resize to a fixed size
            images.append(img)
            labels.append(row['IDENTITY'])  # Use the corresponding label
            
    # Convert to numpy arrays
    images = np.array(images).reshape(-1, image_size[0], image_size[1], 1) / 255.0  # Normalize
    labels = np.array(labels)
    return images, labels

In [15]:
# Step 2: Preprocessing
def preprocess_labels(labels, label_type="char"):
    if label_type == "char":
        # Convert letters to numerical classes
        unique_labels = sorted(list(set(labels)))
        label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
        labels_encoded = np.array([label_to_index[label] for label in labels])
        labels_categorical = to_categorical(labels_encoded, num_classes=len(unique_labels))
        return labels_categorical, label_to_index
    return labels, None

In [16]:
# Step 3: Build the Model
def build_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

In [ ]:
# Step 4: Train and Test the Model
def train_and_test(train_folder, train_csv, test_folder, test_csv, image_size=(64, 64)):
    # Load train and test data
    print("Loading training data...")
    X_train, y_train = load_data(train_folder, train_csv, image_size)
    print("Loading test data...")
    X_test, y_test = load_data(test_folder, test_csv, image_size)

    # Preprocess labels
    y_train, label_map = preprocess_labels(y_train, label_type="char")
    y_test, _ = preprocess_labels(y_test, label_type="char")
    
    # Build the model
    num_classes = y_train.shape[1]
    model = build_model(input_shape=(image_size[0], image_size[1], 1), num_classes=num_classes)

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)
    
    # Evaluate on the test set
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

    return model, label_map

In [18]:
# Train and evaluate the model
model, label_map = train_and_test(train_folder, train_csv, test_folder, test_csv)

Loading training data...
Loading test data...


g:\GitHub\Handwritten_Image_Recognition\model\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


MemoryError: Unable to allocate 3.13 GiB for an array with shape (41370, 20280) and data type float32

In [19]:
# Save the trained model
model.save("handwritten_letter_recognition_model.h5")
print("Model saved successfully.")

NameError: name 'model' is not defined

In [ ]:
import matplotlib.pyplot as plt

def visualize_samples(images, labels, label_map=None, num_samples=25):
    plt.figure(figsize=(10, 10))
    for i in range(num_samples):
        plt.subplot(5, 5, i + 1)
        plt.imshow(images[i].reshape(images[i].shape[0], images[i].shape[1]), cmap='gray')
        plt.title(f"Label: {labels[i] if label_map is None else list(label_map.keys())[np.argmax(labels[i])]}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Example usage after loading data
X_train, y_train = load_data(train_folder, train_csv)
y_train_categorical, label_map = preprocess_labels(y_train, label_type="char")
visualize_samples(X_train, y_train_categorical, label_map)

In [ ]:
def plot_training_history(history):
    # Plot loss
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the function after training
plot_training_history(history)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(model, X_test, y_test, label_map):
    # Get predictions
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    # Generate confusion matrix
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    labels = list(label_map.keys())  # Class names
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

    # Plot confusion matrix
    plt.figure(figsize=(10, 10))
    disp.plot(cmap=plt.cm.Blues, values_format='d', ax=plt.gca())
    plt.title("Confusion Matrix")
    plt.show()

# Call the function after evaluation
plot_confusion_matrix(model, X_test, y_test, label_map)

In [ ]:
def visualize_predictions(model, X_test, y_test, label_map, num_samples=10):
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    plt.figure(figsize=(15, 5))
    for i in range(num_samples):
        plt.subplot(2, 5, i + 1)
        plt.imshow(X_test[i].reshape(X_test[i].shape[0], X_test[i].shape[1]), cmap='gray')
        predicted_label = list(label_map.keys())[y_pred_classes[i]]
        true_label = list(label_map.keys())[y_true_classes[i]]
        plt.title(f"True: {true_label}\nPred: {predicted_label}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Call the function after testing
visualize_predictions(model, X_test, y_test, label_map)